###Abstract ai jobs proyect:
El presente proyecto se centra en el análisis de un dataset de ofertas laborales en el área de IA, que incluye información sobre título del puesto, ubicación, experiencia requerida, tipo de contrato, nivel de trabajo remoto, tamaño de la empresa, habilidades solicitadas, educación requerida, industria, beneficios y salario en dólares estadounidenses.

La elección de este dataset responde al interés de comprender los factores que influyen en la remuneración de profesionales del sector, un área donde las competencias técnicas, la localización y la experiencia pueden tener un peso significativo. El objetivo principal es identificar patrones y relaciones entre las características del puesto y el salario ofrecido, y sentar las bases para un modelo predictivo que permita estimar el salario en función de dichas variables.

El análisis se desarrollará en varias etapas: identificación y tratamiento de valores perdidos, exploración de las distribuciones y relaciones entre variables, creación de visualizaciones multivariadas y cálculo de medidas estadísticas. Este enfoque permitirá responder preguntas clave como:


*   ¿qué nivel de experiencia está mejor remunerado?
*   ¿cuál es la influencia del trabajo remoto?
*   ¿importa más la localización de la empresa o la residencia del empleado?


Los hallazgos obtenidos no solo aportarán información útil para quienes buscan oportunidades laborales en IA, sino también para empresas que desean establecer bandas salariales competitivas. En futuras fases, el proyecto evolucionará hacia la construcción de un modelo para la predicción de salarios, integrando las conclusiones obtenidas en esta etapa inicial.


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.compose import make_column_selector

In [ ]:
# Defino el dataframe a partir de la url y separo las features del target
url = 'https://raw.githubusercontent.com/ignaciogualini/Proyecto_Coder_DS1_Gualini/refs/heads/main/datasets/ai_job_dataset.csv'
ai_jobs_df = pd.read_csv(url, index_col='job_id')
target = 'salary_usd'
X, y = (ai_jobs_df.drop(columns=target), ai_jobs_df[target])
ai_jobs_df.head()

### Preguntas de investigación:

1. **¿Cómo afecta la experiencia al salario?**
2. **¿Qué impacto tiene el tamaño de la empresa en el salario?**
3. **¿Existe alguna relación entre el salario y el trabajo remoto?**
4. **¿Por qué algunos salarios son extremadamente altos y qué factores los explican?¿Son outliers?**
---
### Hipótesis:

*  H1. A mayor experiencia, mayor será el salario.
*  H2. Las empresas grandes (`L`) ofrecen salarios más altos que las medianas (`M`) y pequeñas (`S`).
*  H3. Sería de esperar que esta variable (`remote_ratio`) no tenga peso alguno en el salario.
*  H4. Los salarios extremadamente altos corresponden a roles senior o especializados, con muchos años de experiencia y alto nivel de `experience_level`.

In [ ]:
print(f'El dataframe a analizar se compone de {ai_jobs_df.shape[0]} filas y {ai_jobs_df.shape[1]} columnas\n')
print(f'La cantidad de valores nulos por columna es:\n\n {ai_jobs_df.isnull().sum()}')

In [ ]:
# Separo las columnas numéricas de las categóricas
num_col_selector = make_column_selector(dtype_exclude=object)
cat_col_selector = make_column_selector(dtype_include=object)
num_cols = num_col_selector(X)
cat_cols = cat_col_selector(X)
X_num = X[num_cols]
X_cat = X[cat_cols]
print(f'De estas {ai_jobs_df.shape[1]} columnas, 1 es el target (salario en USD), {len(num_cols)} son numéricas y {len(cat_cols)} son categóricas')

In [ ]:
print(f'También podemos realizar una breve descripción estadística de sus columnas numéricas:\n\n{X_num.describe()}')
print(f'\nPodemos observar que algunas de las columnas numéricas poseen valores mucho mayores a otras,\npor lo que no es descabellado pensar en una futura estandarización.')
print(f'\nEn cuanto al target:\n\n{y.describe()}')

In [ ]:
# Creo una matriz de correlación para el mapa de calor
correlation = ai_jobs_df[num_cols + [target]].corr()
_ = sns.heatmap(correlation, cmap='Reds', annot=True, linewidths=.5, linecolor='black')
print('Gráfico 1')

Tras realizar un análisis de correlación entre variables numéricas, se puede observar que ninguna de ellas guarda relación más que consigo misma. Por lo que no tenemos columnas numéricas que sean redundantes para el análisis.

También se ve que el target (`salary_usd`) posee una correlación fuerte con los años de experiencia. Por lo que se confirma que es una buena hipótesis a plantear y a resolver en los siguientes gráficos.

In [ ]:
print(f'La variable remote_ratio solo posee {X_num["remote_ratio"].nunique()} valores únicos ({X_num["remote_ratio"].unique()}), por lo que podríamos usar esta informacion en un gráfico.\n')

# Categorizo la variable "company_size" para que aparezca en el orden deseado de tamaño
X['company_size'] = pd.Categorical(X['company_size'], categories=['S', 'M', 'L'], ordered=True)
print(f'Por otro lado, la variable "company_size" posee los valores {list(X["company_size"].cat.categories)}\n')

print('Gráfico 2.1/Gráfico 2.2')

color_map = {'S': 'black', 'M': 'blue', 'L': 'red'}

# Divido el gráfico en dos ejes separados en dos columnas
fig, (ax1, ax2) = plt.subplots(ncols=2, figsize=(12, 6))

# Línea de tendencia para ax1
sns.regplot(
    ax=ax1,
    data=ai_jobs_df,
    x='years_experience',
    y=target,
    scatter=False,
    line_kws={'color': 'black'},
    label='línea de tendencia',
    )

# Gráfico de dispersión para ax1
sc1 = ax1.scatter(
    X_num['years_experience'],
    y,
    c=X_num['remote_ratio'],
    cmap='plasma',
    alpha=1,
    edgecolors='w',
    linewidth=0.5,
)
ax1.set_xlabel('Años de experiencia')
ax1.set_ylabel('Salario en USD')
ax1.set_title('Relación Experiencia vs Salario con Remote Ratio')
plt.colorbar(sc1, label='Remote Ratio')
ax1.legend()

# Línea de tendencia para ax2
sns.regplot(
    ax=ax2,
    data=ai_jobs_df,
    x='years_experience',
    y=target,
    scatter=False,
    line_kws={'color': 'black'},
    label='línea de tendencia',
    )

# Gráfico de dispersión para ax2
sc2 = ax2.scatter(
    X_num['years_experience'],
    y,
    c=X['company_size'].map(color_map),
    alpha=0.8
)
ax2.set_xlabel('Años de experiencia')
ax2.set_ylabel('Salario en USD')
ax2.set_title('Relación Experiencia vs Salario con Tamaño de empresa')
plt.colorbar(sc2, label='Company size')
ax2.legend()

plt.tight_layout()
plt.show()

Efectivamente, existe una relación positiva entre los años de experiencia de un trabajador y su salario.
En cuanto a la variable `remote_ratio`, podemos ver que en cada columna del gráfico a la izquierda (2.1) los colores se distribuyen de forma variada, lo cual indica que el Remote Ratio no está tan correlacionado directamente con el salario.

Realizando este mismo análisis en el gráfico de la derecha (2.2), en cada columna se puede observar cierta tendencia: los puntos rojos suelen estar más arriba y los de color negro abajo, por lo cual existe cierta relación positiva entre el salario de un trabajador y el tamaño de la empresa (`company_size`). Entendiendo que mientras más grande sea la empresa, mejor va a pagar a sus empleados.

In [ ]:
variables = ['job_description_length', 'benefits_score', 'years_experience'] # Defino variables de interés para la función de figura
print('Gráfico 3')

# Paso a formato largo para que mi FacetGrid lo entienda
df_long = X.melt(
    id_vars=['company_size'],
    value_vars=variables,
    var_name='variable',
    value_name='valor'
)

# Utilizo una función de figura donde las columnas se representen por el tamaño de la compañía
# En cada fila se encuentran las variables de interés
g = sns.FacetGrid(
    df_long,
    row='variable',
    col='company_size',
    margin_titles=True,
    sharex=False,
    sharey=False
)

g.map(sns.histplot, 'valor', bins=20, color='blue', edgecolor='black', kde=True)

g.set_titles(row_template='{row_name}', col_template='{col_name}')
g.set_axis_labels('', "Frecuencia")
plt.show()

Observando los histogramas, podemos concluir lo siguiente:
la distribución de cada variable numérica (exceptuando "remote ratio") es independiente al tamaño de empresa.
Por lo que estas variables no están sesgadas en función a si una empresa es grande, mediana o chica.

Este tipo de análisis resulta conveniente en estas situaciones, ya que cuando dos variables o features están correlacionadas fuertemente aportan información redundante, y complican la interpretación del modelo.
Un claro ejemplo de esto son las columnas `experience_level` y `years_experience`, ya que apuntan a lo mismo.

In [ ]:
print('Gráfico 4.1/Gráfico 4.2')
sns.set_style('darkgrid')

# Divido el gráfico en dos ejes separados en dos columnas
fig, (ax1, ax2) = plt.subplots(nrows=1, ncols=2, figsize=(14, 6), sharey=False)

# Elaboro un boxplot para ax1
sns.boxplot(data=ai_jobs_df, y=target, ax=ax1)
ax1.axhline(y=y.mean(), color='red', linestyle='--', linewidth=2, label=f'Salario medio: ${y.mean():.2f}')
ax1.axhline(y=y.mean() - y.std(), color='blue', linestyle='--', linewidth=2, label=f'Límite inferior: ${(y.mean() - y.std()):.2f}')
ax1.axhline(y=y.mean() + y.std(), color='green', linestyle='--', linewidth=2, label=f'Límite superior: ${(y.mean() + y.std()):.2f}')
ax1.set_ylabel('Salario en USD')
ax1.set_title('Boxplot')
ax1.legend(bbox_to_anchor=(1.05, 1), loc='upper left')

# Elaboro un histograma para ax2
sns.histplot(data=ai_jobs_df, x=target, ax=ax2, bins=20, edgecolor='black', kde=True)
ax2.axvline(x=y.mean(), color='red', linestyle='--', linewidth=2,)
ax2.axvline(x=y.mean() - y.std(), color='blue', linestyle='--', linewidth=2,)
ax2.axvline(x=y.mean() + y.std(), color='green', linestyle='--', linewidth=2,)
ax2.set_xlabel('Salario en USD')
ax2.set_ylabel('Cantidad')
ax2.set_title('Histograma')
plt.xticks(rotation=45)

_ = plt.suptitle('Distribución de salarios')

plt.tight_layout(rect=[0, 0, 0.95, 1])

plt.show()

print('\nSe puede ver en el gráfico de caja que tenemos muchos valores por fuera de 1.5IQR. Esto puede llegar a indicar que son outliers.')
print('Si analizamos el histograma, vemos que hay una distribución sesgada a la derecha típica de variables salariales:\nla mayoría gana entre un rango bajo/medio, pero hay una minoría con salarios muy altos.')
print('Entonces se puede decir que estos valores son parte natural de la distribución y no un error de registro. Son casos válidos y aportan información relevante para el posterior modelo.')
print(f'Esto también va de la mano con el valor de la desviación estándar: {y.std():.1f} $. En proporción, es el {((y.std() / y.mean()) * 100):.2f} % del salario medio, indicando un valor considerable para este parámetro.\nEsto se relaciona con la distribución sesgada a la derecha, ya que hay muchos valores lejanos a la media.\n')

###Conclusiones preliminares en función de las hipótesis:

*  ***H1. A mayor experiencia, mayor será el salario:***
el mapa de calor (*Gráfico 1*) así como la gráfica de dispersión y la línea de regresión (*Gráficos 2.1 y 2.2*) confirman la relación positiva que guardan estas dos variables, confirmando la hipótesis n° 1✅.

*  ***H2. Las empresas grandes (`L`) ofrecen salarios más altos que las medianas (`M`) y pequeñas (`S`):***
En el segundo gráfico de dispersión (*Gráfico 2.2*) se puede observar que en cada columna los puntos rojos suelen estar más arriba y los de color negro abajo, por lo cual existe cierta relación positiva entre el salario de un trabajador y el tamaño de la empresa. Esto confirma la hipótesis n° 2✅.

*  ***H3. Sería de esperar que esta variable (`remote_ratio`) no tenga peso alguno en el salario:***
Los colores por `remote_ratio` en el primer gráfico de dispersión (*Gráfico 2.1*) no muestran un patrón claro ni correlación fuerte con el salario.
Esto sugiere que el nivel de trabajo remoto no es un factor determinante por sí solo para el salario. Hipótesis confirmada✅.

*  ***H4. Los salarios extremadamente altos corresponden a roles senior o especializados, con muchos años de experiencia y alto nivel de `experience_level`:***
Este apartado va de la mano con la primera hipótesis. En el boxplot (*Gráfico 4.1*) podemos ver que **estadísticamente** tenemos varios outliers. Pero si realizamos un análisis en conjunto con el histograma (*Gráfico 4.2*) los valores altos no son outliers ni errores de medición, son una representación fiel de una distribución sesgada a la derecha. Esta brecha salarial (y a su vez, desviación estándar considerable) es común en este tipo de empleos donde existen muchos factores que generen diferencia en cuanto al pago.
